# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = os.path.join("..", "WeatherPy", "Output", "output.csv")
original_weather_df = pd.read_csv(csv_path)
original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]] = original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].astype(float)
original_weather_df.dropna(axis=0,how='all', inplace=True)#drop all NaN items from data frame
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vuktyl,63.86,57.31,0.90,93.0,25.0,7.27,RU,1.608250e+09
1,Mlonggo,-6.53,110.70,77.00,94.0,40.0,3.36,ID,1.608250e+09
3,Hermanus,-34.42,19.23,62.06,73.0,8.0,12.68,ZA,1.608250e+09
4,Rikitea,-23.12,-134.97,77.67,74.0,35.0,16.13,PF,1.608250e+09
6,Atuona,-9.80,-139.03,78.82,72.0,35.0,16.24,PF,1.608250e+09
...,...,...,...,...,...,...,...,...,...
617,Manzhouli,49.60,117.43,-8.28,85.0,0.0,11.07,CN,1.608250e+09
618,Bilibino,68.05,166.44,-14.76,85.0,15.0,4.27,RU,1.608250e+09
619,Havelock,34.88,-76.90,45.00,90.0,30.0,5.32,US,1.608250e+09
620,Ferrol,43.48,-8.24,57.99,72.0,61.0,10.00,ES,1.608250e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps using my Google API key
gmaps.configure(api_key=g_key)

coordinates = original_weather_df[['Lat', 'Lng']].astype(float)
humidity = original_weather_df["Humidity"].astype(float)
# Customize the size of the figure
figure_layout = {
    'width': '533px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [4]:
good_weather_df = original_weather_df.copy()

good_weather_df = good_weather_df[(good_weather_df['Max Temp'] > 70) & (good_weather_df['Max Temp'] < 80)] #only show cities with a max temperature between 70 and 80 degrees
good_weather_df = good_weather_df[good_weather_df['Wind Speed'] < 10] #and only show cities where the wind speed is <10 mph
good_weather_df = good_weather_df[good_weather_df['Cloudiness'] == 0] #and only show cities where there's no clouds
good_weather_df.dropna(axis=0,how='any', inplace=True) #removing any of the cities that have null values

good_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
157,Caravelas,-17.71,-39.25,74.86,86.0,0.0,5.95,BR,1.608250e+09
185,Oussouye,12.48,-16.55,73.40,94.0,0.0,4.70,SN,1.608250e+09
196,Suleja,9.18,7.18,75.20,50.0,0.0,4.70,NG,1.608249e+09
344,Chirongui,-12.93,45.15,77.00,88.0,0.0,3.36,YT,1.608250e+09
435,Tahoua,14.89,5.27,75.20,21.0,0.0,9.17,NE,1.608250e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df  = good_weather_df.copy()
hotel_df["Hotel Name"] = "NaN" #initialize Hotel Name column

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():#using iterrows to iterate through pandas dataframe
    
    cityName = row['City'] #get city name type from df
    lat = row['Lat'] #get latitude from df
    lng = row['Lng'] #get longitude from df
    
    params = {
    "location": f"{lat},{lng}", #substituting latitude and longitude into the API call
    "rankby": "distance",
    "type": "lodging", #this type is what a hotel would be classified under according to Google API docs
    "distance": 5000, #within 5000 meters of this location
    "key": g_key,
    }    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cityName}.")
    response = requests.get(base_url, params=params).json()    
    
    results = response['results'] #extract results
    
    try:
        print(f"Closest {cityName} lodging is {results[0]['name']}.") #find the closest hotel to coordinates  
        #add the name field of the first hotel from the JSON response to the Hotel Name column in my data frame
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 157: Caravelas.
Closest Caravelas lodging is Pousada Tropical.
------------
Retrieving Results for Index 185: Oussouye.
Closest Oussouye lodging is Campement ALULUM.
------------
Retrieving Results for Index 196: Suleja.
Closest Suleja lodging is Cosco Guest Inn..
------------
Retrieving Results for Index 344: Chirongui.
Closest Chirongui lodging is La Villa Mahatsara.
------------
Retrieving Results for Index 435: Tahoua.
Closest Tahoua lodging is Bungalows Guesthouse.
------------


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
###i still need to add the names of the hotel, city, and country to the pins on the map###. 
#I also still need to do the summary of my findings at the top of the document

# Customize the size of the figure
figure_layout = {
    'width': '533px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()

#Add marker layer ontop of heat map
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info #fill in hotel name and information on the text box popup when you click on the pin on the map
)
# Add the layer to the map
fig.add_layer(markers)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))